## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/home/yuzjzj/miniconda3/envs/t2/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(32,(3,3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 8s 162us/sample - loss: 1.4376 - accuracy: 0.4973
Epoch 2/100
50000/50000 [==============================] - 6s 126us/sample - loss: 1.0235 - accuracy: 0.6406
Epoch 3/100
50000/50000 [==============================] - 6s 126us/sample - loss: 0.8736 - accuracy: 0.6923
Epoch 4/100
50000/50000 [==============================] - 6s 126us/sample - loss: 0.7693 - accuracy: 0.7303
Epoch 5/100
50000/50000 [==============================] - 6s 126us/sample - loss: 0.6749 - accuracy: 0.7641
Epoch 6/100
50000/50000 [==============================] - 6s 127us/sample - loss: 0.5852 - accuracy: 0.7974
Epoch 7/100
50000/50000 [==============================] - 6s 126us/sample - loss: 0.4979 - accuracy: 0.8287
Epoch 8/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.4184 - accuracy: 0.8556
Epoch 9/100
50000/50000 [==============================] - 6s 126us/sample - loss: 0.3407 - accuracy: 0.8

50000/50000 [==============================] - 6s 125us/sample - loss: 0.0222 - accuracy: 0.9931
Epoch 76/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0200 - accuracy: 0.9936
Epoch 77/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0212 - accuracy: 0.9936
Epoch 78/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0169 - accuracy: 0.9945
Epoch 79/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0137 - accuracy: 0.9953
Epoch 80/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0186 - accuracy: 0.9938
Epoch 81/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0221 - accuracy: 0.9934
Epoch 82/100
50000/50000 [==============================] - 6s 125us/sample - loss: 0.0193 - accuracy: 0.9937
Epoch 83/100
50000/50000 [==============================] - 6s 124us/sample - loss: 0.0234 - accuracy: 0.9927
Epoch 84/100
50000/5000

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.2872600e-11, 0.0000000e+00, 1.0000000e+00, 1.1122531e-08,
        2.1406092e-09, 0.0000000e+00, 1.9445413e-33, 0.0000000e+00,
        2.1867329e-32, 0.0000000e+00]], dtype=float32)